<a href="https://colab.research.google.com/github/MortezaMahdaviMortazavi/EmotionRecognition/blob/master/EmoPars_Emotion_Detection_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
from torch.utils.data import DataLoader
from transformers import AdamW
import torch.nn as nn

In [3]:
batch_size = 16
learning_rate = 1e-5
num_epochs = 3

In [4]:
# Load the pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')


In [5]:
from transformers import XLMRobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup

In [7]:
test = pd.read_csv('test_cleaned_emopars.csv', index_col=0)

In [10]:
train = pd.read_csv('train_cleaned_emopars.csv', index_col=0)

In [11]:
train.head()

,Unnamed: 0,text,Anger,Fear,Happiness,Hatred,Sadness,Wonder,primary_emotion
0,0,کرونا رو شکست میدهیم مرحله بعد چه گوهی میخوای ...,4,3,1,3,3,4,0
1,2,یکی از پدرسوختگی های برانداز اینه که ظاهرا ژست...,5,3,2,4,5,2,0
2,3,یکی از دوستای دبستانم,1,0,0,1,0,0,6
3,4,اینقدر گرفتار مسایل میشی که تخصص از دستت درد ...,2,1,0,1,0,4,5
4,5,شماهایی که توییتها رو میبینید و سکوت میکنید شم...,2,0,0,3,3,3,3


'Anger':0, 'Fear':1, 'Happiness':2, 'Hatred':3, 'Sadness':4, 'Wonder':5, 'Other':6

In [8]:
test = test.drop(['primary_emotion', 'Unnamed: 0'], axis=1)
test.head()

,text,Anger,Fear,Happiness,Hatred,Sadness,Wonder
0,من خیلی خودسانسوری می‌کنم تو اینستا هر چی فالو...,1,1,2,1,0,2
1,بعد اتمام جلسه مجلس روند بازار برگشت بورس,3,0,1,2,1,1
2,کاربران توییتر در جریان طوفان توییتری اعتراض ب...,4,0,0,2,2,0
3,وحشی شدن معده بعد از رسیدن به ایران اجتناب ناپ...,1,0,1,0,0,1
4,سحام نیوز بیانیه مشترک عربستان و امارات با پرو...,0,0,0,1,1,0


In [12]:
train = train.drop(['primary_emotion', 'Unnamed: 0'], axis=1)
train.head()

,text,Anger,Fear,Happiness,Hatred,Sadness,Wonder
0,کرونا رو شکست میدهیم مرحله بعد چه گوهی میخوای ...,4,3,1,3,3,4
1,یکی از پدرسوختگی های برانداز اینه که ظاهرا ژست...,5,3,2,4,5,2
2,یکی از دوستای دبستانم,1,0,0,1,0,0
3,اینقدر گرفتار مسایل میشی که تخصص از دستت درد ...,2,1,0,1,0,4
4,شماهایی که توییتها رو میبینید و سکوت میکنید شم...,2,0,0,3,3,3


In [9]:
targets = ["Anger", "Fear", "Happiness", "Hatred", "Sadness", "Wonder"]

Preprocess

In [13]:
def preprocess(df):
    # Normalize emotion labels
    emotion_columns = ['Anger', 'Fear', 'Happiness', 'Hatred', 'Sadness', 'Wonder']

    for col in emotion_columns:
        df[col] = df[col] / df[col].max()  # Normalize to the range [0, 1]

    # Apply threshold for binary labels
    threshold = 0.35
    for col in emotion_columns:
        df[col] = df[col].apply(lambda x: 1 if x >= threshold else 0)

    return df

In [14]:
train = preprocess(train)
test = preprocess(test)

In [15]:
X_train, y_train = train['text'].values.tolist(), train[targets].values.tolist()
X_test, y_test = test['text'].values.tolist(), test[targets].values.tolist()

In [16]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, texts, labels, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt'
        )

        inputs = {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.float32)  # Use the provided numeric label directly
        }

        return inputs


In [17]:
train_dataset = TextDataset(tokenizer,X_train,y_train,max_length=128)
test_dataset = TextDataset(tokenizer,X_test,y_test,max_length=128)

In [18]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [19]:
import torch.nn as nn

# Define the loss function
criterion = nn.BCEWithLogitsLoss()

In [20]:
# Check if CUDA (GPU) is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [27]:
from transformers import AutoModel

In [34]:
class XLMRobertaGRUClassifier(nn.Module):
    def __init__(self, num_classes):
        super(XLMRobertaGRUClassifier, self).__init__()
        self.num_classes = num_classes
        self.xlmroberta = AutoModel.from_pretrained("xlm-roberta-base")

        # Add a GRU layer
        self.gru = nn.GRU(self.xlmroberta.config.hidden_size, hidden_size=self.xlmroberta.config.hidden_size, num_layers=1, batch_first=True)

        # # Correct the hidden size for the linear layer
        self.linear = nn.Linear(self.xlmroberta.config.hidden_size, num_classes)  # Multiply by 2 for bidirectional GRU
        self.dropout = nn.Dropout(0.2)

    def forward(self, input_ids, attention_mask):
        outputs = self.xlmroberta(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
        logits = outputs.last_hidden_state

        # Pass the logits through the GRU layer
        gru_output, _ = self.gru(logits)

        logits = self.linear(self.dropout(gru_output[:, -1, :]))
        return logits

In [35]:
# Assuming num_classes and your dataset are defined
num_classes = 6  # Change this to your actual number of classes
learning_rate = 1e-5
batch_size = 16
epochs = 3

model = XLMRobertaGRUClassifier(num_classes)
model.to(device)  # Move model to GPU if available

XLMRobertaGRUClassifier(
  (xlmroberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=Tru

In [26]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from transformers import XLMRobertaTokenizer, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm

In [36]:
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [38]:
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        logits = model(input_ids, attention_mask)
        loss = criterion(logits, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Optional gradient clipping
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        print(f"batch loss: {loss.item():.4f}")

    average_loss = total_loss / len(train_dataloader)
    print('-' * 80)
    print(f"Epoch {epoch + 1} - Average Loss: {average_loss:.4f}")

# Save your trained model
torch.save(model.state_dict(), 'xlmroberta_gru_model.pth')


Epoch 1:   0%|          | 1/1688 [00:00<14:21,  1.96it/s]

batch loss: 0.5995


Epoch 1:   0%|          | 2/1688 [00:00<12:58,  2.17it/s]

batch loss: 0.6122


Epoch 1:   0%|          | 3/1688 [00:01<12:36,  2.23it/s]

...

batch loss: 0.4961


Epoch 3: 100%|█████████▉| 1687/1688 [12:29<00:00,  2.25it/s]

batch loss: 0.4065


Epoch 3: 100%|██████████| 1688/1688 [12:30<00:00,  2.25it/s]

batch loss: 0.5387
--------------------------------------------------------------------------------
Epoch 3 - Average Loss: 0.4582


In [ ]:
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}"):
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        logits = model(input_ids, attention_mask)
        loss = criterion(logits, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Optional gradient clipping
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        print(f"batch loss: {loss.item():.4f}")

    average_loss = total_loss / len(train_dataloader)
    print('-'*80)
    print(f"Epoch {epoch + 1} - Average Loss: {average_loss:.4f}")

# Save your trained model
torch.save(model.state_dict(), 'xlmroberta_gru_model.pth')


In [ ]:
# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        logits = model(input_ids, attention_mask).logits
        loss = criterion(logits, labels)

        loss.backward()
        optimizer.step()

    print("Epoch:", epoch, "loss:", loss.item())


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch: 0 loss: 0.45309561491012573
Epoch: 1 loss: 0.5078866481781006
Epoch: 2 loss: 0.4640763998031616


In [ ]:
model.save_pretrained("emotion_detection_model")

In [40]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

model.eval()
with torch.no_grad():
    all_true_labels = []
    all_predicted_labels = []

    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        logits = model(input_ids, attention_mask)

        # Apply sigmoid activation to logits to get predicted probabilities
        predicted_probs = torch.sigmoid(logits)

        # Apply a threshold (e.g., 0.5) to convert probabilities to binary predictions
        threshold = 0.5
        predicted_labels = (predicted_probs > threshold).to(torch.float32)

        # Convert labels and predictions to numpy arrays
        true_labels = labels.cpu().numpy()
        predicted_labels = predicted_labels.cpu().numpy()

        # Append the true and predicted labels to the lists
        all_true_labels.extend(true_labels)
        all_predicted_labels.extend(predicted_labels)


In [41]:
true_labels = all_true_labels
predicted_labels = all_predicted_labels

In [42]:
import numpy as np

In [43]:
# Calculate accuracy for each label
label_accuracies = []
for i in range(len(true_labels)):
    label_accuracy = accuracy_score(true_labels[i], predicted_labels[i])
    label_accuracies.append(label_accuracy)

# Calculate macro F1 score for each label and take the average
label_macro_f1_scores = []
for i in range(len(true_labels)):
    label_macro_f1 = f1_score(true_labels[i], predicted_labels[i], average='macro')
    label_macro_f1_scores.append(label_macro_f1)

# Calculate the average accuracy and macro F1 score
average_accuracy = np.mean(label_accuracies)
average_macro_f1 = np.mean(label_macro_f1_scores)

print("Average Accuracy:", average_accuracy)
print("Average Macro F1 Score:", average_macro_f1)

Average Accuracy: 0.8223333333333334
Average Macro F1 Score: 0.6198228956228956


In [44]:
test_data = pd.read_csv('3. data_emotion_without_label.csv')

In [45]:
test_data.head()

,local_id,tweet
0,973588328225411072,لنگ پولیس ۱ برق شیراز دوبی ۰\nخداییش تیم...
1,983807604982996995,دوستان بارسایی. \nحالا بشینید و فوتبال خوب تما...
2,1378698455305060353,والا ۱۳ روز #عید که هیچی با این شرایط یک سال ا...
3,1380360990793953282,بدبختی ماگناه #بیگانه نبود\nپیوندمن وشما #صمیم...
4,1382562339258441728,تصور کن انقدر #عشق بدی تاهمه چیز اطرافت شاخ و ...


In [46]:
model = model.to('cpu')

In [48]:
# Tokenize and predict emotions for each tweet
results = []

for j in range(len(test_data)):
    data_point = test_data.iloc[j]
    text = data_point["tweet"]
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)

    with torch.no_grad():
        outputs = model(**inputs)
    probabilities = torch.sigmoid(outputs)

    # Find the emotion with the highest predicted value
    primary_emotion_index = torch.argmax(probabilities)
    primary_emotion = targets[primary_emotion_index]

    # Normalize and apply threshold
    threshold = 0.15
    predictions = (probabilities > threshold).cpu().numpy().tolist()[0]

    # Create a dictionary with the required information
    result = {
        "local_id": data_point["local_id"],
        "tweet": text,
        "primary_emotion": primary_emotion,
    }

    # Add emotion predictions to the dictionary
    for i, emotion in enumerate(targets):
        result[emotion] = predictions[i]

    results.append(result)

# Create a DataFrame
result_df = pd.DataFrame(results)


In [49]:
targets = ["Anger", "Fear", "Happiness", "Hatred", "Sadness", "Wonder"]

In [51]:
result_df.head()

,local_id,tweet,primary_emotion,Anger,Fear,Happiness,Hatred,Sadness,Wonder
0,973588328225411072,لنگ پولیس ۱ برق شیراز دوبی ۰\nخداییش تیم...,Happiness,False,False,True,False,False,False
1,983807604982996995,دوستان بارسایی. \nحالا بشینید و فوتبال خوب تما...,Happiness,False,False,True,False,False,True
2,1378698455305060353,والا ۱۳ روز #عید که هیچی با این شرایط یک سال ا...,Sadness,False,False,True,False,True,False
3,1380360990793953282,بدبختی ماگناه #بیگانه نبود\nپیوندمن وشما #صمیم...,Sadness,True,True,False,True,True,False
4,1382562339258441728,تصور کن انقدر #عشق بدی تاهمه چیز اطرافت شاخ و ...,Happiness,False,False,True,False,True,False


In [52]:
result_df.to_csv('resulr.csv')

In [54]:
# Mapping from primary emotions
primary_emotion_mapping = {"Anger":'anger', "Sadness":'sadness', "Wonder":'surprise', "Happiness":'happiness', "Fear":'fear', "Hatred":'disgust', "Other":'other'}

# Tokenize and predict emotions for each tweet
results = []

for j in range(len(test_data)):
    data_point = test_data.iloc[j]
    text = data_point["tweet"]
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    probabilities = torch.sigmoid(outputs)

    # Find the emotion with the highest predicted value
    primary_emotion_index = torch.argmax(probabilities)
    primary_emotion = primary_emotion_mapping[targets[primary_emotion_index]]

    # Normalize and apply threshold
    threshold = 0.15
    predictions = (probabilities > threshold).cpu().numpy().tolist()[0]

    # Create a dictionary with the required information
    result = {
        "local_id": data_point["local_id"],
        "tweet": text,
        "primary_emotion": primary_emotion,
    }

    # Add emotion predictions to the dictionary
    for i, emotion in enumerate(targets):
        # Convert True/False to 1/0
        result[emotion] = int(predictions[i])

    # If all predicted scores are 0, set 'other' as primary emotion
    if all(score == 0 for score in predictions):
        result["primary_emotion"] = "Other"

    results.append(result)

# Create a DataFrame
result_df = pd.DataFrame(results)

# Print the DataFrame
print(result_df)


                local_id                                              tweet  \
0     973588328225411072  لنگ پولیس  ۱      برق شیراز دوبی ۰\nخداییش تیم...   
1     983807604982996995  دوستان بارسایی. \nحالا بشینید و فوتبال خوب تما...   
2    1378698455305060353  والا ۱۳ روز #عید که هیچی با این شرایط یک سال ا...   
3    1380360990793953282  بدبختی ماگناه #بیگانه نبود\nپیوندمن وشما #صمیم...   
4    1382562339258441728  تصور کن انقدر #عشق بدی تاهمه چیز اطرافت شاخ و ...   
..                   ...                                                ...   
495  1644747933890539521  <@USERNAME> با توجه به علاقه شدید جامعه سلبرید...   
496  1644749245377904640                    یک همدم باوفا ندیدم جز درد ...    
497  1644758722047266817  جناب،#رضاکیانیان اگربدنبال قاتل احتمالی مرحوم،...   
498  1644768661633396736         مرا به میکده بَر،\n در خُمِ شراب انداز...    
499  1644770853035339780  خواجه چقدر غم‌انگیز فرمود:\n\nنسیمِ زلفِ تو گر...   

    primary_emotion  Anger  Fear  Happiness  Hatred

In [55]:
result_df.head()

,local_id,tweet,primary_emotion,Anger,Fear,Happiness,Hatred,Sadness,Wonder
0,973588328225411072,لنگ پولیس ۱ برق شیراز دوبی ۰\nخداییش تیم...,happiness,0,0,1,0,0,0
1,983807604982996995,دوستان بارسایی. \nحالا بشینید و فوتبال خوب تما...,happiness,0,0,1,0,0,1
2,1378698455305060353,والا ۱۳ روز #عید که هیچی با این شرایط یک سال ا...,sadness,0,0,1,0,1,0
3,1380360990793953282,بدبختی ماگناه #بیگانه نبود\nپیوندمن وشما #صمیم...,sadness,1,1,0,1,1,0
4,1382562339258441728,تصور کن انقدر #عشق بدی تاهمه چیز اطرافت شاخ و ...,happiness,0,0,1,0,1,0


In [56]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   local_id         500 non-null    int64 
 1   tweet            500 non-null    object
 2   primary_emotion  500 non-null    object
 3   Anger            500 non-null    int64 
 4   Fear             500 non-null    int64 
 5   Happiness        500 non-null    int64 
 6   Hatred           500 non-null    int64 
 7   Sadness          500 non-null    int64 
 8   Wonder           500 non-null    int64 
dtypes: int64(7), object(2)
memory usage: 35.3+ KB


In [57]:
# Replace 'Other' with 'other' in the 'primary_emotion' column
result_df['primary_emotion'] = result_df['primary_emotion'].replace({'Other': 'other'})

result_df.head()

,local_id,tweet,primary_emotion,Anger,Fear,Happiness,Hatred,Sadness,Wonder
0,973588328225411072,لنگ پولیس ۱ برق شیراز دوبی ۰\nخداییش تیم...,happiness,0,0,1,0,0,0
1,983807604982996995,دوستان بارسایی. \nحالا بشینید و فوتبال خوب تما...,happiness,0,0,1,0,0,1
2,1378698455305060353,والا ۱۳ روز #عید که هیچی با این شرایط یک سال ا...,sadness,0,0,1,0,1,0
3,1380360990793953282,بدبختی ماگناه #بیگانه نبود\nپیوندمن وشما #صمیم...,sadness,1,1,0,1,1,0
4,1382562339258441728,تصور کن انقدر #عشق بدی تاهمه چیز اطرافت شاخ و ...,happiness,0,0,1,0,1,0


In [58]:
# Rename columns
result_df = result_df.rename(columns={"Anger":'anger', "Sadness":'sadness', "Wonder":'surprise', "Happiness":'happiness', "Fear":'fear', "Hatred":'disgust'})

# Print the reordered DataFrame
result_df.head()


,local_id,tweet,primary_emotion,anger,fear,happiness,disgust,sadness,surprise
0,973588328225411072,لنگ پولیس ۱ برق شیراز دوبی ۰\nخداییش تیم...,happiness,0,0,1,0,0,0
1,983807604982996995,دوستان بارسایی. \nحالا بشینید و فوتبال خوب تما...,happiness,0,0,1,0,0,1
2,1378698455305060353,والا ۱۳ روز #عید که هیچی با این شرایط یک سال ا...,sadness,0,0,1,0,1,0
3,1380360990793953282,بدبختی ماگناه #بیگانه نبود\nپیوندمن وشما #صمیم...,sadness,1,1,0,1,1,0
4,1382562339258441728,تصور کن انقدر #عشق بدی تاهمه چیز اطرافت شاخ و ...,happiness,0,0,1,0,1,0


In [59]:
cols = ['local_id', 'tweet', 'primary_emotion', 'anger', 'disgust', 'fear', 'sadness', 'happiness', 'surprise']

In [60]:
# Reorder the columns
result_df1 = result_df[cols]

result_df1.head()

,local_id,tweet,primary_emotion,anger,disgust,fear,sadness,happiness,surprise
0,973588328225411072,لنگ پولیس ۱ برق شیراز دوبی ۰\nخداییش تیم...,happiness,0,0,0,0,1,0
1,983807604982996995,دوستان بارسایی. \nحالا بشینید و فوتبال خوب تما...,happiness,0,0,0,0,1,1
2,1378698455305060353,والا ۱۳ روز #عید که هیچی با این شرایط یک سال ا...,sadness,0,0,0,1,1,0
3,1380360990793953282,بدبختی ماگناه #بیگانه نبود\nپیوندمن وشما #صمیم...,sadness,1,1,1,1,0,0
4,1382562339258441728,تصور کن انقدر #عشق بدی تاهمه چیز اطرافت شاخ و ...,happiness,0,0,0,1,1,0


In [62]:
result_df1.to_csv('result_final_AE.csv')

In [ ]:
PATH = "model.pt"

torch.save({
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, PATH)